### 数据加载

In [1]:
import pandas as pd

train_df = pd.read_csv('Data/Movie_RS.csv')
print(train_df.shape)
train_df.head(1)

(10000, 13)


,ID,Movie_Name,Movie_Score,Review_Count,Movie_Star_Distribution,Collect_Date,Username,Post_Date,Score,User_Comment,User_Comment_Distribution,Comment_Like_Count,Movie_Tags
0,0,"1988年的妮可 Nico, 1988",7.5,565,15.2%48.2%32.3%3.4%0.8%,2019-10-05,尾黑,2018-06-23,3,成本低廉的PPT电影，用Nico生命中最后一年发生的事给Nico的歌配上情节，倒不算尴尬。女...,66%31%3%,4,"['音乐', '电影', '儿子', '丝绒', '人物', '传记', '传记片', '歌..."


### 数据预处理

提取电影信息

In [2]:
train_df.drop_duplicates(
    subset=['Movie_Name'], keep='first', inplace=True)

# 打印电影的数据结构
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26 entries, 0 to 9848
Data columns (total 13 columns):
ID                           26 non-null int64
Movie_Name                   26 non-null object
Movie_Score                  26 non-null float64
Review_Count                 26 non-null int64
Movie_Star_Distribution      26 non-null object
Collect_Date                 26 non-null object
Username                     26 non-null object
Post_Date                    26 non-null object
Score                        26 non-null int64
User_Comment                 26 non-null object
User_Comment_Distribution    26 non-null object
Comment_Like_Count           26 non-null int64
Movie_Tags                   26 non-null object
dtypes: float64(1), int64(4), object(8)
memory usage: 2.8+ KB


推荐质量控制

In [3]:
# 设置电影打分阈值 Threshold
Threshold = 7.5
train_df = train_df[train_df['Movie_Score'] > Threshold]
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12 entries, 159 to 8811
Data columns (total 13 columns):
ID                           12 non-null int64
Movie_Name                   12 non-null object
Movie_Score                  12 non-null float64
Review_Count                 12 non-null int64
Movie_Star_Distribution      12 non-null object
Collect_Date                 12 non-null object
Username                     12 non-null object
Post_Date                    12 non-null object
Score                        12 non-null int64
User_Comment                 12 non-null object
User_Comment_Distribution    12 non-null object
Comment_Like_Count           12 non-null int64
Movie_Tags                   12 non-null object
dtypes: float64(1), int64(4), object(8)
memory usage: 1.3+ KB


In [4]:
# 建立用户名和 id 映射的字典
user_dict = {value: index for index,
             value in enumerate(train_df['Username'].unique())}
# 开始对原数据进行转换
train_df['uid_int'] = train_df['Username'].apply(lambda x: user_dict[x])


# 建立电影名和 id 映射的字典
item_dict = {value: index for index, 
             value in enumerate(train_df['Movie_Name'].unique())}
# 开始对原数据进行转换
train_df['item_int'] = train_df['Movie_Name'].apply(lambda x: item_dict[x])


# 用户 id 字典翻转
reverse_user_dict = {v: k for k, v in user_dict.items()}
# 电影 id 字典翻转
reverse_item_dict = {v: k for k, v in item_dict.items()}

设置电影和用户特征。

In [5]:
# 电影特征
items_f = ['Movie_Score', 'Review_Count', 'item_int', 'Movie_Name','Movie_Tags']

# 用户特征
users_f = ['uid_int','User_Comment_Distribution','Username']

### 基于标签的最近邻电影推荐

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer

# 调节最低词频 min_df 可以提高准确度，但是同时也改变了召回
content_vec = TfidfVectorizer(min_df=5)

#### 标签向量化

In [7]:
from sklearn.metrics.pairwise import cosine_similarity

# 文本向量化
vec_matrix = content_vec.fit_transform(train_df['Movie_Tags'])

# 计算相似度计算矩阵
%time cosine_sim_result = cosine_similarity(vec_matrix)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 802 µs


假设当前电影 id，并打印详细信息。

In [8]:
# 输入电影名字
movie_realname = '一曲难忘 A Song to Remember'

# 打印电影信息
train_df.loc[train_df['Movie_Name'].isin([movie_realname])].drop_duplicates(
    'item_int', keep='first', inplace=False)[items_f].head()

,Movie_Score,Review_Count,item_int,Movie_Name,Movie_Tags
2973,7.8,2079,5,一曲难忘 A Song to Remember,"['电影', '音乐', '老师', '教授', '音乐课', '时候', '艺术', '有..."


In [9]:
# 定义近邻检索函数
def get_movie_recommendations(movie_realname, cosine_sim, topk):
    idx = item_dict[movie_realname]
    
    # 去除自身的计算结果
    sim_scores = sorted(enumerate(cosine_sim[idx]), key=lambda x: x[1], reverse=True)
    return sim_scores[1:topk+1]

通过建立的电影 id 向量，找到最近的 tokn 个电影。

In [10]:
topn = 5

# 开始进行进行近邻推荐,输出电影 id 和近邻计算的分数
%time ANN_List = get_movie_recommendations(movie_realname, cosine_sim_result, topn)
ANN_List

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 35.8 µs


[(8, 0.831335667568609),
 (6, 0.8112590212923482),
 (0, 0.7783721074571646),
 (10, 0.7783721074571646),
 (1, 0.7563206116569436)]

打印近邻的电影信息。

In [11]:
# 数据去重，保留首先出现的数据。
ANN_List_Info = train_df.loc[train_df['item_int'].isin(
    [i[0] for i in ANN_List])].drop_duplicates('item_int', keep='first', inplace=False)

# 打印近邻检索的数据
ANN_List_Info[items_f].head()

,Movie_Score,Review_Count,item_int,Movie_Name,Movie_Tags
159,7.9,1509,0,24小时狂欢派对 24 Hour Party People,"['音乐', '纪录片', '电影', '时候', '演员', '片子', '工厂', '乐..."
373,7.9,12803,1,42号传奇 42,"['成就', '励志', '桥段', '夫妻', '棒球', '电影', '核心', '数字..."
3679,7.9,4395,6,一轮明月,"['电影', '大师', '人物', '感觉', '人生', '主旋律', '流水账', '..."
4771,7.7,3760,8,万夫莫敌 Invincible,"['励志', '电影', '橄榄球', '故事', '时候', '球队', '男人', '影..."
5992,7.6,550,10,上帝的使者 الرسالة,"['宗教', '电影', '历史', '先知', '影片', '主角', '信仰', '镜头..."


#### 应用推广
主要根据标签之间的相似度进行近邻检索进行推荐的一种方式。

1. 用户电影之间的电影近邻推荐。
2. 用户用户之间的用户近邻推荐。